In [1]:
import os

In [2]:
%pwd

'd:\\model deploy end to end\\heartfailml\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\model deploy end to end\\heartfailml'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from heartfailure.constants import *
from heartfailure.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from heartfailure import logger
from heartfailure.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-09-28 23:35:10,032: INFO: common: yaml file: D:\model deploy end to end\heartfailml\config\config.yaml loaded successfully]
[2023-09-28 23:35:10,039: INFO: common: yaml file: D:\model deploy end to end\heartfailml\params.yaml loaded successfully]
[2023-09-28 23:35:10,042: INFO: common: yaml file: D:\model deploy end to end\heartfailml\schema.yaml loaded successfully]
[2023-09-28 23:35:10,046: INFO: common: created directory at: artifacts1]
[2023-09-28 23:35:10,049: INFO: common: created directory at: artifacts1/data_ingestion]


[2023-09-28 23:35:11,334: INFO: 1434958058: artifacts1/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 3536
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "7b14a90e1a8b6c43b7f26afab0c71930fc33dcdedf1b549d6334634c865a14f7"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 0CFE:25788:20D2A9:252E7F:6515A5FD
Accept-Ranges: bytes
Date: Thu, 28 Sep 2023 18:05:11 GMT
Via: 1.1 varnish
X-Served-By: cache-bom4734-BOM
X-Cache: HIT
X-Cache-Hits: 1
X-Timer: S1695924311.418382,VS0,VE342
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: fc20b863ffbcb39ffcc6025855f58c4c1d675b6d
Expires: Thu, 28 Sep 2023 18:10:11 GMT
Source-Age: 0

]
